In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 0. Préalable
Chargement des données

In [ ]:
covid_0 = pd.read_excel('covid.xlsx')

In [ ]:
# on réalise une copie de notre dataset pour ne pas l'altérer durant notre étude
covid = covid_0.copy()

## 1. Exploratory Data Analysis (EDA)

### 1.1. Analyse de forme

Bref aperçu du contenu du dataset

In [ ]:
covid.sample(5) # visualiser un échantillon aléatoire du dataset

Dimension du dataset

In [ ]:
covid.shape # dimension du dataset (nombre de lignes, nombre de colonnes)

Analyse du type de chaque variable

In [ ]:
# pd.set_option('display.max_row', 111)
covid.dtypes # retourne le type de chaque variable

In [ ]:
covid.dtypes.value_counts()

Analyse des valeurs manquantes

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(covid.isna(), cbar=False)

In [ ]:
# covid.isna().sum()
(covid.isna().sum() * 100 / covid.shape[0]).sort_values()

> Résumé de l'analyse de forme

1. **Variable target :** SARS-Cov-2 exam result
2. **Dimension du dataset :** (5644, 111)
3. **Types des variables :** 70 qualitatives et 41 quantitatives
4. **Identification des valeurs manquantes :** 
- beaucoup de Nan (moitié des variables > 90% de NaN)
- 2 groupes de données : <br>
76% -> tests viraux (bactéries, virus)<br>
89% -> tests sanguins (cellules)<br>

### 1.2. Analyse de fond

Elimination des colonnes inutiles

In [ ]:
covid.dropna(axis=1, how='all', inplace=True) # suppression des colonnes vides
covid.shape

In [ ]:
covid = covid[covid.columns[covid.isna().sum() / covid.shape[0] < 0.9]] # suppression de toutes les colonnes avec plus de 90% de valeurs manquantes
covid.shape

In [ ]:
covid.drop('Patient ID', axis=1, inplace=True) # suppression de l'identifiant des patients

In [ ]:
covid.sample(3)

Visualisation de la target

In [ ]:
# covid['SARS-Cov-2 exam result'].value_counts()
covid['SARS-Cov-2 exam result'].value_counts(normalize=True) * 100

Histogramme des variables continues

In [ ]:
# visualisation des variables flottantes
for colonne in covid.select_dtypes('float'):
	plt.figure()
	sns.distplot(covid[colonne])

In [ ]:
sns.distplot(covid['Patient age quantile'], bins=20) # visualisation de l'age

Visualisation des variables discrètes

In [ ]:
for colonne in covid.select_dtypes('object'):
	print(f'{colonne :-<30} {covid[colonne].unique()}')

In [ ]:
for colonne in covid.select_dtypes('object'):
	plt.figure()
	covid[colonne].value_counts().plot.pie()

Création de sous-ensembles positifs et négatifs

In [ ]:
positive_covid = covid[covid['SARS-Cov-2 exam result']	== 'positive']
negative_covid = covid[covid['SARS-Cov-2 exam result']	== 'negative']

Création des sous-ensembles Virus et Sang

In [ ]:
missing_rate = covid.isna().sum() / covid.shape[0]
blood_columns = covid.columns[(missing_rate < 0.9) & (missing_rate > 0.88)]
viral_columns = covid.columns[(missing_rate < 0.87) & (missing_rate > 0.75)]

Relation target/blood

In [ ]:
for colonne in blood_columns:
	plt.figure()
	sns.distplot(positive_covid[colonne], label='positive')
	sns.distplot(negative_covid[colonne], label='negative')
	plt.legend()

Relation target/age

In [ ]:
sns.countplot(data=covid, x='Patient age quantile', hue='SARS-Cov-2 exam result')

Relation target/viral

In [ ]:
for colonne in viral_columns:
	plt.figure()
	sns.heatmap(pd.crosstab(covid['SARS-Cov-2 exam result'], covid[colonne]), annot=True, fmt='d')

> Résumé de l'analyse de fond

1. **Visualisation de la target :** 10% de cas positifs
2. **Signification des variables :** 
- variables continues standardisées, skewed (asymétriques), test sanguin
- age quantile : graphique difficile à interpréter car les données ont été traitées et la nature de ce traitement est inconnue
- variables qualitatives : binaires, virales, Rhinovirus très varié
3. **Visualisation des relations features/target :** 
- target/blood : les taux de leucocytes, monocytes et plaquettes semblent liés au covid 19
- target/age : les individus d'age faible sont très peu contaminés
- target/viral : les doubles maladies sont très rares. Rhinovirus/Enterovirus positif => covid 19 positif ?

> Conclusions initiales de l'EDA<br>
> - Beacoup de données manquantes (au mieux, on garde 20% du dataset)
> - 2 groupes de données intéressantes (viral et sanguin)
> - Presque pas de variable discriminante pour distinguer les cas positifs/négatifs.

### Analyse plus détaillée

Relations variables/variables

Relations taux sanguin

In [ ]:
sns.heatmap(covid[blood_columns].corr())

Relations age/sang

In [ ]:
for colonne in blood_columns:
	plt.figure()
	sns.lmplot(data=covid, x='Patient age quantile', y=colonne, hue='SARS-Cov-2 exam result')

In [ ]:
covid.corr()['Patient age quantile'].sort_values()

Relations viral/viral

In [ ]:
pd.crosstab(covid['Influenza A'], covid['Influenza A, rapid test'])

In [ ]:
pd.crosstab(covid['Influenza B'], covid['Influenza B, rapid test'])

Relations viral/sanguin

Création d'une nouvelle variable "est_malade"

In [ ]:
covid["is_sick"] = np.sum(covid[viral_columns[:-2]] == 'detected', axis=1) >= 1 
covid.sample(3)

In [ ]:
sick_rows = covid[covid['is_sick'] == True]
healthy_rows = covid[covid['is_sick'] == False]

In [ ]:
covid.sample(3)

In [ ]:
for colonne in blood_columns:
	plt.figure()
	sns.distplot(sick_rows[colonne], label='sick')
	sns.distplot(healthy_rows[colonne], label='healthy')
	plt.legend()

Relation hospitalisation/malade

In [ ]:
def hospitalisation(df: pd.DataFrame):
	if df['Patient addmited to regular ward (1=yes, 0=no)'] == 1:
		return 'surveillance'
	elif df['Patient addmited to semi-intensive unit (1=yes, 0=no)'] == 1:
		return 'soins semi-intensifs'
	elif df['Patient addmited to intensive care unit (1=yes, 0=no)'] == 1:
		return 'soins intensifs'
	else:
		return 'inconnu'

In [ ]:
covid['statut'] = covid.apply(hospitalisation, axis=1)

In [ ]:
covid.sample(3)

In [ ]:
for colonne in blood_columns:
	plt.figure()
	for cat in covid['statut'].unique():
		sns.distplot(covid[covid['statut'] == cat][colonne], label=cat)
	plt.legend()

Relations variables/variables :
- blood_data/blood_data : certaines variables sont très corrélées (+0.9)
- blood_data/age : très faible corrélation entre l'age et le taux sanguin
- viral/viral : le test rapide de grippe donne de mauvais résultats. Il faudra peut-être la laisser tomber
- relation maladie/blood_data : les taux sanguins entre malades et covid 19 sont différents
- relation hospitalisation/malade : 
- relation hospitalisation/blood : intéressant dans le cas où on voudrait prédire dans quel service un patient devrait aller

Analyse NaN : viral ~ 1350 (92/8), blood ~ 600 (86/14), both (90/10)

Hypothèses de départ :
- Les individus atteints du covid 19 ont des taux de leucocytes, monocytes, plaquettes significativement différents
- Soit H0 = Les taux moyens sont égaux chez les individus positifs et négatifs
- Les individus atteints d'une quelonque maladie ont des taux significativement différents

### T-Test

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
print(positive_covid.shape, negative_covid.shape)
# les deux tableaux ne comportent pas le même nombre de lignes

In [ ]:
balance_neg = negative_covid.sample(positive_covid.shape[0])

In [ ]:
def t_test(colonne):
	alpha = 0.02
	stat, p = ttest_ind(balance_neg[colonne].dropna(), positive_covid[colonne].dropna())
	if p < alpha:
		return 'H0 rejetée'
	else:
		return 'H0 acceptée'

In [ ]:
for colonne in blood_columns:
	print(f'{colonne :-<50} {t_test(colonne)}')

H0 rejetée